In [ ]:
library("knitr")
library("glmnet")
library("ggplot2")
library("tidyverse")
library("tidymodels")
library("data.table")

## Lasso-penalised logistic regression using tidymodels

For this illustration, we will use the metabolomics dataset on NMR spectroscopy metabolite abundances in diabetes patients (with controls):

In [ ]:
mtbsl1 <- fread("../data/MTBSL1.tsv")
nrow(mtbsl1)

In [ ]:
head(mtbsl1)

In [ ]:
names(mtbsl1)[c(4:ncol(mtbsl1))] <- paste("mtbl",seq(1,ncol(mtbsl1)-3), sep = "_")
table(mtbsl1$Metabolic_syndrome,mtbsl1$Gender)

We see that the metabolites abundances have largely different scales/magnitudes, and it could be a good idea to normalise them before running the model (remember that Lasso constraints the size of the coefficients, and these depends on the magnitude of each variable $\rightarrow$ same $\lambda$ applied to all variables)  

In [ ]:
library("repr")
options(repr.plot.width=14, repr.plot.height=8)

mm_diab <- mtbsl1 %>%
  gather(key = "metabolite", value = "abundance", -c(`Primary ID`,Gender,Metabolic_syndrome))

mm_diab %>%
  ggplot(aes(metabolite, abundance, fill = as.factor(metabolite))) +
  geom_boxplot(show.legend = FALSE)

### Training and test sets

We now split the data in the training and test sets:

In [ ]:
diab_dt <- select(mtbsl1, -c(`Primary ID`, Gender)) ## remove gender for the moment, and keep only mìnumeric features for convinience
mtbsl1_split <- initial_split(diab_dt, strata = Metabolic_syndrome)
mtbsl1_train <- training(mtbsl1_split)
mtbsl1_test <- testing(mtbsl1_split)

### Preprocessing

We remove variables with zero variance (no variability, not informative) and normalise all numeric variables:

In [ ]:
## build a recipe for preprocessing
mtbsl1_rec <- recipe(Metabolic_syndrome ~ ., data = mtbsl1_train) %>%
  # update_role(`Primary ID`, new_role = "ID") %>%
  step_zv(all_numeric(), -all_outcomes()) %>%
  step_normalize(all_numeric(), -all_outcomes())

print(mtbsl1_rec)

In [ ]:
mtbsl1_prep <- mtbsl1_rec %>%
  prep(strings_as_factors = FALSE)

print(mtbsl1_prep)
mtbsl1_train <- juice(mtbsl1_prep)

## The Lasso model

We start by specifying our Lasso model:

- it's logistic regression for a classification problem
- we set the $\lambda$ parameter (penalty) to the arbitrary value of 0.1
- mixture: amount of L1 regularization, when 1 it's Lasso (0 is Ridge regression)
- the engine is set to `glmnet`

Then we add everything to a workflow object, piecewise, and fit the Lasso model:

In [ ]:
lasso_spec <- logistic_reg(mode = "classification", penalty = 0.1, mixture = 1) %>%
  set_engine("glmnet")

In [ ]:
wf <- workflow() %>%
  add_recipe(mtbsl1_rec)

In [ ]:
lasso_fit <- wf %>%
  add_model(lasso_spec) %>%
  fit(data = mtbsl1_train)

In [ ]:
lasso_fit %>%
  pull_workflow_fit() %>%
  tidy()

In [ ]:
lasso_fit %>%
  pull_workflow_fit() %>%
  tidy() %>%
  filter(estimate > 0)

## Tuning the hyperparameters

We use k-fold cross-validation to tune the hyperparameters ($\lambda$ penalty in this case) in the training set:

In [ ]:
diab_cv <- vfold_cv(mtbsl1_train, v=5, repeats = 10, strata = Metabolic_syndrome)

tune_spec <- logistic_reg(penalty = tune(), mixture = 1) %>%
  set_engine("glmnet")

lambda_grid <- grid_regular(penalty(), levels = 50, filter = penalty <= .05)
print(lambda_grid)


In [ ]:
doParallel::registerDoParallel()

lasso_grid <- tune_grid(
  wf %>% add_model(tune_spec),
  resamples = diab_cv,
  grid = lambda_grid
)

In [ ]:
lasso_grid %>%
  collect_metrics()

In [ ]:
lasso_grid %>%
  collect_metrics() %>%
  ggplot(aes(penalty, mean, color = .metric)) +
  geom_errorbar(aes(
    ymin = mean - std_err,
    ymax = mean + std_err
  ),
  alpha = 0.5
  ) +
  geom_line(size = 1.5) +
  facet_wrap(~.metric, scales = "free", nrow = 2) +
  scale_x_log10() +
  theme(legend.position = "none")

In [ ]:
lowest_roc <- lasso_grid %>%
  select_best("roc_auc")

print(lowest_roc)

In [ ]:
final_lasso <- finalize_workflow(
  wf %>% add_model(tune_spec),
  lowest_roc
)

print(final_lasso)

### Variable importance

In [ ]:
library("vip")

final_lasso %>%
  fit(mtbsl1_train) %>%
  pull_workflow_fit() %>%
  vi(lambda = lowest_roc$penalty) %>%
  mutate(
    Importance = abs(Importance),
    Variable = fct_reorder(Variable, Importance)
  ) %>%
  filter(Importance > 0) %>%
  ggplot(aes(x = Importance, y = Variable, fill = Sign)) +
  geom_col() +
  scale_x_continuous(expand = c(0, 0)) +
  labs(y = NULL)

## Testing the model

In [ ]:
lr_res <- last_fit(
  final_lasso,
  mtbsl1_split
) 

lr_res %>%
  collect_metrics()


In [ ]:
lr_res %>% collect_predictions() %>%
  group_by(.pred_class,Metabolic_syndrome) %>%
  summarise(N=n()) %>%
  spread(key = ".pred_class", value = N)

In [ ]:
mtbsl1_testing <- mtbsl1_prep %>% bake(testing(mtbsl1_split)) ## preprocess test data

final_lasso_fit <- fit(final_lasso, data = mtbsl1_train) ## fit final model on the training set

final_lasso_fit %>%
  predict(mtbsl1_testing, type = "class") %>%
  bind_cols(mtbsl1_testing) %>%
  # metrics(truth = Metabolic_syndrome, estimate = .pred_class)
  group_by(.pred_class,Metabolic_syndrome) %>%
  summarise(N=n()) %>%
  spread(key = ".pred_class", value = N)

In [ ]:
lr_auc <- 
  lr_res %>% 
  collect_predictions() %>% 
  roc_curve(Metabolic_syndrome,`.pred_Control Group`) %>% 
  mutate(model = "Logistic Regression")

autoplot(lr_auc)

## Add gender (factor) to the Lasso model

To add factors to the Lasso model (engine `glmnet`) we need to use [one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)

In [ ]:
diab_dt <- model.matrix(~ ., mtbsl1[,"Gender"]) %>% as_tibble() %>% select(GenderMale) %>% rename(gender = GenderMale) %>% 
bind_cols(diab_dt)

head(diab_dt)

#### Interactive exercise

We'll now rerun a Lasso-penalised logistic regression model including Gender as feature, going together through the steps involved in Lasso models with `tidymodels`:

1. splitting the data
2. preprocessing
3. hyperparameters tuning
4. fitting the final model
5. predictions on the test set

In [ ]:
## code here


In [ ]:
## code here